# generate summary

In [ ]:
import sys
sys.path.append("/GFM") # your project root directory
from GraphCLIP.generate_summary_api import *
from openai import OpenAI
# api_key = 'your_api_key'
# client = OpenAI(api_key=api_key)
client = None

In [2]:
class Args:
    def __init__(self, task_name):
        self.model = "graphclip"
        self.model_id = "exp1"
        self.task_name = task_name
        self.input_dim = 128
        self.compress_function = "none"
        self.cache_compress = False
        self.seed = 0
        self.is_logging = False
        self.num_workers = 0
        self.device = None
        self.pattern = "simple"

args = Args(task_name="pretrain")
pretrain_exps = {
    "exp1": pretrain,
    "exp2": pretrain,
    "exp3": pretrain_exp3,
    "exp4": pretrain_exp4,
}
pretrain_dict = pretrain_exps[args.model_id]
# recommmed run one by one
pretrain_dict = {
    'Cora': cora,
    # 'ACM': acm,
    # 'DBLP': dblp,
    # 'Reddit': reddit,
    # 'Texas': texas,
    # 'Wisconsin': wisconsin,
    # 'Cornell': cornell,
    # 'IMDB': imdb,
    
    # 'Photo': photo,
    # 'Computers': computers,
    # 'Amazon': amazon,
    # 'Amazon-HeTGB': amazonh,
    # 'HIV': hiv,
    # 'COX2': cox2,
    # 'PROTEINS': proteins,
    # 'ENZYMES': enzymes,
    # 'FB15K-237': fb15k237,
    # 'NELL': nell,

    # # large-scale datasets
    # 'ogbn-arxiv': ogbn_arxiv,
    # 'Elliptic': elliptic
}

In [ ]:
main_request(args, pretrain_dict, client)

In [5]:
save_path = f"{ROOT_DIR}/GraphCLIP/api/output_dict.json"
with open(save_path, "r", encoding="utf-8") as f:
    output_dict = json.load(f)

In [16]:
# check status
for name in list(output_dict.keys()):
    if name in ['FB15K-237']:
        print(f"############ Dataset: {name} ##############")
        for batch_id in output_dict[name]["batch_ids"]:
            print(f"Batch ID: {batch_id} \n"
                f"Status: {client.batches.retrieve(batch_id).status} "
                f"Request Counts: {client.batches.retrieve(batch_id).request_counts}")

############ Dataset: FB15K-237 ##############
Batch ID: batch_69495d71f0d481909545f2c2bc2f00de 
Status: completed Request Counts: BatchRequestCounts(completed=6000, failed=0, total=6000)
Batch ID: batch_69495dc6ee088190ba693153810ba34e 
Status: completed Request Counts: BatchRequestCounts(completed=6000, failed=0, total=6000)
Batch ID: batch_69495dec7878819099e66099e6e15fdc 
Status: completed Request Counts: BatchRequestCounts(completed=2505, failed=0, total=2505)


In [17]:
# if failed, retry
name = "FB15K-237"
get_fail_requests(client, name)
retry_failed_requests(client, name)

[INFO] Total retry requests: 0


['/home/shenghua/Graph-Foundation-Library/datasets/FB15K-237/preprocess/graphclip/retry_0.jsonl']

In [18]:
# generate final summary
for name in output_dict.keys():
    if name in ['FB15K-237']:
        merge_batch_summaries(
            item_list_path=output_dict[name]["item_list_path"],
            batch_ids=output_dict[name]["batch_ids"],
            summary_path=output_dict[name]["summary_path"],
            client=client
        )

Loaded 6000 results from batch batch_69495d71f0d481909545f2c2bc2f00de
input_tokens=11810217, avg=1968.37
output_tokens=2431769, avg=405.29
Loaded 6000 results from batch batch_69495dc6ee088190ba693153810ba34e
input_tokens=11829813, avg=1971.64
output_tokens=2419807, avg=403.30
Loaded 2505 results from batch batch_69495dec7878819099e66099e6e15fdc
input_tokens=4912955, avg=1961.26
output_tokens=1017221, avg=406.08
Merged summaries saved to /home/shenghua/Graph-Foundation-Library/datasets/FB15K-237/preprocess/graphclip/summary.json


### some tools

In [ ]:
# list recent batches
batches = client.batches.list()
for b in batches.data[:10]:
    print(f"ID: {b.id}, Status: {b.status}, Model: {b.model}, {b.request_counts}, usage: {b.usage.output_tokens}")

In [ ]:
# check batch status
client.batches.retrieve('batch_69417358f81481908a1ec508f785607c')

In [ ]:
# cancel batch
client.batches.cancel('batch_693e726592b08190b1c4e651d5588f3e')

In [ ]:
# error check
file_id = client.batches.retrieve('batch_6946da0915ec81909dfa81ed2a96cbe5').error_file_id
result_bytes = client.files.content(file_id)
result_lines = result_bytes.text.splitlines()
result_lines

In [ ]:
# check all summaries added
save_path = f"{ROOT_DIR}/GraphCLIP/api/output_dict.json"
with open(save_path, "r", encoding="utf-8") as f:
    output_dir = json.load(f)

for name in output_dir.keys():
    check_summary(name)

# generate node feature

In [4]:
import sys
sys.path.append("/home/shenghua/Graph-Foundation-Library")
from GraphCLIP.generate_feature import *

In [6]:
class Args:
    def __init__(self, task_name):
        self.model = "graphclip"
        self.model_id = "exp3"
        self.task_name = task_name
        self.pattern = "simple"
        self.input_dim = 384
        self.compress_function = "none"
        self.cache_compress = False
        self.seed = 0
        self.is_logging = False
        self.num_workers = 0
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.checkpoints = str(ROOT_DIR / "checkpoints")

args = Args(task_name="pretrain")
pretrain_exps = {
    "exp1": pretrain,
    "exp2": pretrain,
    "exp3": pretrain_exp3,
    "exp4": pretrain_exp4,
}
pretrain_dict = pretrain_exps[args.model_id]
exp = ExpPretrainGraphCLIP(args, pretrain_dict)

Compression function is None.
Set random seed to 0
Pretrain datasets: Cora, ACM, DBLP
> Loading existing embeddings from: /home/shenghua/Graph-Foundation-Library/datasets/Cora/preprocess/graphclip/node_feature.pt
> Loading existing embeddings from: /home/shenghua/Graph-Foundation-Library/datasets/ACM/preprocess/graphclip/node_feature.pt
> Loading existing embeddings from: /home/shenghua/Graph-Foundation-Library/datasets/DBLP/preprocess/graphclip/node_feature.pt
Finished generating node features for all datasets.


In [22]:
from GraphCLIP.utils.process import parse_source_data
all_source_graph = []
for name, data in exp.pretrain_dict.items():
    source_graph = parse_source_data(name, data)
    all_source_graph.extend(source_graph)

process FB15K-237


100%|██████████| 14505/14505 [00:15<00:00, 910.29it/s]


# generate target sub

In [1]:
import sys
sys.path.append("/home/shenghua/Graph-Foundation-Library")
from gen_target_subg import *
from GraphCLIP.utils.args import Arguments
from data_provider import *
args = Arguments().parser.parse_args([])
args.model = "graphclip"
args.model_id = "exp2"
args.exp_id = "exp2"
args.task_name = "graph"
args.seed = 0
args.is_logging = False
args.compress_function = "none"
args.input_dim = 384
args.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
args.sampler = 'rw'#'khop'
args.walk_step = 256
args.batch_size = 32768
pretrain_exps = {
    "exp1": pretrain,
    "exp2": pretrain,
    "exp3": pretrain_exp3,
    "exp4": pretrain_exp4,
}
pretrain_dict = pretrain_exps[args.model_id]

exps = {
    f"exp{i}": {
        "node": globals()[f"NC_exp{i}"],
        "edge": globals()[f"EC_exp{i}"],
        "graph": globals()[f"GC_exp{i}"],
    }
    for i in range(0, 5)
}
for name, dataset in exps[args.exp_id][args.task_name].items():
    exp = ExpDownstreamBatchGraphCLIP(args, pretrain_dict, name, dataset)

/home/shenghua/anaconda3/envs/py310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Compression function is None.
Set random seed to 0
Target dataset: PROTEINS
Average degree: 3.73
Is undirected: True
> Loading existing embeddings from: /home/shenghua/Graph-Foundation-Library/datasets/PROTEINS/preprocess/graphclip/node_feature.pt
Finished generating node features for PROTEINS dataset.
Loading pre-generated subgraphs...
Loaded pre-generated subgraphs from /home/shenghua/Graph-Foundation-Library/datasets/PROTEINS/preprocess/graphclip/sub_list(none).pt. And added PE feature.
